<a href="https://colab.research.google.com/github/nicolasvargaszz/-PyTorch_intro/blob/main/pytorch_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Definir hiperparámetros
batch_size = 64
learning_rate = 0.01
num_epochs = 10

# Preparar los datos (MNIST)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Definir la red neuronal
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = x.view(-1, 28*28)  # Aplanar las imágenes de 28x28 a un vector de 784
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.softmax(self.fc3(x))
        return x

# Inicializar el modelo, la función de pérdida y el optimizador
model = SimpleNN()
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Entrenamiento del modelo
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Evaluación del modelo
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on the test set: {100 * correct / total:.2f}%")


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 16285877.28it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 512253.65it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4547944.81it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2065545.78it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch [1/10], Loss: 0.8183
Epoch [2/10], Loss: 0.3197
Epoch [3/10], Loss: 0.2653
Epoch [4/10], Loss: 0.2299
Epoch [5/10], Loss: 0.2024
Epoch [6/10], Loss: 0.1796
Epoch [7/10], Loss: 0.1610
Epoch [8/10], Loss: 0.1453
Epoch [9/10], Loss: 0.1312
Epoch [10/10], Loss: 0.1205
Accuracy on the test set: 96.16%


In [ ]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Azure Text Analytics Configuration
endpoint = "YOUR_AZURE_ENDPOINT"  # Replace with your Azure endpoint
key = "YOUR_AZURE_KEY"  # Replace with your Azure key

# Azure Text Analytics client setup
def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
        endpoint=endpoint, credential=ta_credential)
    return text_analytics_client

# Function to analyze sentiment using Azure
def sentiment_analysis(client, text):
    response = client.analyze_sentiment(documents=[text])[0]
    return response.sentiment

# Sample text for sentiment analysis
texts = ["I love programming with PyTorch.", "Machine learning is hard.", "Azure services are very powerful!"]

# Authenticate Azure client
client = authenticate_client()

# Analyze sentiments
sentiments = [sentiment_analysis(client, text) for text in texts]

# Convert sentiment into numeric form for ML model training
sentiment_map = {'positive': 1, 'neutral': 0, 'negative': -1}
labels = [sentiment_map[sent] for sent in sentiments]

# Data preparation for PyTorch model
data = torch.tensor([[i] for i in range(len(texts))], dtype=torch.float32)
labels = torch.tensor(labels, dtype=torch.float32).view(-1, 1)

# PyTorch Model: Simple Neural Network
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

model = SimpleModel()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
epochs = 50
for epoch in range(epochs):
    model.train()

    # Forward pass
    outputs = model(data)
    loss = criterion(outputs, labels)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss for every 10 epochs
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Predictions after training
model.eval()
predictions = model(data).detach().numpy()

# Plot results using Matplotlib
plt.plot(data.numpy(), labels.numpy(), 'ro', label="True Sentiments")
plt.plot(data.numpy(), predictions, 'b-', label="Predicted Sentiments")
plt.title("Sentiment Analysis Prediction (Azure + PyTorch)")
plt.xlabel("Text Samples")
plt.ylabel("Sentiment (Positive=1, Neutral=0, Negative=-1)")
plt.legend()
plt.show()
